In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn
import math
%matplotlib inline
import os

import sklearn
import scipy.io as sio
from numba import jit
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
from astropy.timeseries import LombScargle
import re

In [21]:

                                                                 
for j in range(1):                                                                  #action
    path = os.path.abspath(os.getcwd())+'\\sensor_csv'+'\\'+str(i)+'\\a'+str(j+1)
    file = os.listdir(path)
    file1 = str(file)
    print(file1)  
    file1 = file1.replace("[", "")
    file1 = file1.replace("]", "")
    file1 = file1.replace("'", "")
    print(file1) 
    filelist = file1.split(",")
    
    

#print(files)


['cx_t1.csv', 'cx_t10.csv', 'cx_t11.csv', 'cx_t12.csv', 'cx_t13.csv', 'cx_t2.csv', 'cx_t3.csv', 'cx_t4.csv', 'cx_t5.csv', 'cx_t6.csv', 'cx_t7.csv', 'cx_t8.csv', 'cx_t9.csv', 'cyy_t1.csv', 'cyy_t10.csv', 'cyy_t2.csv', 'cyy_t3.csv', 'cyy_t4.csv', 'cyy_t5.csv', 'cyy_t6.csv', 'cyy_t7.csv', 'cyy_t8.csv', 'cyy_t9.csv', 'myj_t1.csv', 'myj_t10.csv', 'myj_t2.csv', 'myj_t3.csv', 'myj_t4.csv', 'myj_t5.csv', 'myj_t6.csv', 'myj_t7.csv', 'myj_t8.csv', 'myj_t9.csv', 'qyh_t1.csv', 'qyh_t10.csv', 'qyh_t11.csv', 'qyh_t2.csv', 'qyh_t3.csv', 'qyh_t4.csv', 'qyh_t5.csv', 'qyh_t6.csv', 'qyh_t7.csv', 'qyh_t8.csv', 'qyh_t9.csv', 'zyh_t1.csv', 'zyh_t2.csv', 'zyh_t3.csv', 'zyh_t4.csv', 'zyh_t5.csv', 'zyh_t6.csv', 'zyh_t7.csv', 'zyh_t8.csv', 'zyh_t9.csv']
cx_t1.csv, cx_t10.csv, cx_t11.csv, cx_t12.csv, cx_t13.csv, cx_t2.csv, cx_t3.csv, cx_t4.csv, cx_t5.csv, cx_t6.csv, cx_t7.csv, cx_t8.csv, cx_t9.csv, cyy_t1.csv, cyy_t10.csv, cyy_t2.csv, cyy_t3.csv, cyy_t4.csv, cyy_t5.csv, cyy_t6.csv, cyy_t7.csv, cyy_t8.csv, cyy_t9

In [31]:
mat_filenames = []
filenames = []
for file1 in filelist:
    x = re.split("_|\.",file1)
    #print(x)
    name = x[0]
    time = x[1]
    #print(type(file))
    for j in range(22):
        str1 = name+"_a"+str(j+1)+"_"+time+".mat"
        filename = name+"_a"+str(j+1)+"_"+time
        mat_filenames.append(str1)
        filenames.append(filename)


In [32]:
#print(files)
print(len(filelist))
print(len(mat_filenames))
print(len(filenames))

53
1166
1166


In [34]:
def spectral_entropy(x, sf=100, method='fft', nperseg=None, normalize=False,axis=-1):
    """Spectral Entropy.
    Parameters
    ----------
    x : list or np.array
        1D or N-D data.
    sf : float
        Sampling frequency, in Hz.
    method : str
        Spectral estimation method:
        * ``'fft'`` : Fourier Transform (:py:func:`scipy.signal.periodogram`)
        * ``'welch'`` : Welch periodogram (:py:func:`scipy.signal.welch`)
    nperseg : int or None
        Length of each FFT segment for Welch method.
        If None (default), uses scipy default of 256 samples.
    normalize : bool
        If True, divide by log2(psd.size) to normalize the spectral entropy
        between 0 and 1. Otherwise, return the spectral entropy in bit.
    axis : int
        The axis along which the entropy is calculated. Default is -1 (last).
    Returns
    -------
    se : float
        Spectral Entropy
    Notes
    -----
    Spectral Entropy is defined to be the Shannon entropy of the power
    spectral density (PSD) of the data:
    .. math:: H(x, sf) =  -\\sum_{f=0}^{f_s/2} P(f) \\log_2[P(f)]
    Where :math:`P` is the normalised PSD, and :math:`f_s` is the sampling
    frequency.
    References
    ----------
    - Inouye, T. et al. (1991). Quantification of EEG irregularity by
      use of the entropy of the power spectrum. Electroencephalography
      and clinical neurophysiology, 79(3), 204-210.
    - https://en.wikipedia.org/wiki/Spectral_density
    - https://en.wikipedia.org/wiki/Welch%27s_method
    Examples
    --------
    Spectral entropy of a pure sine using FFT
    >>> import numpy as np
    >>> import entropy as ent
    >>> sf, f, dur = 100, 1, 4
    >>> N = sf * dur # Total number of discrete samples
    >>> t = np.arange(N) / sf # Time vector
    >>> x = np.sin(2 * np.pi * f * t)
    >>> np.round(ent.spectral_entropy(x, sf, method='fft'), 2)
    0.0
    Spectral entropy of a random signal using Welch's method
    >>> np.random.seed(42)
    >>> x = np.random.rand(3000)
    >>> ent.spectral_entropy(x, sf=100, method='welch')
    6.980045662371389
    Normalized spectral entropy
    >>> ent.spectral_entropy(x, sf=100, method='welch', normalize=True)
    0.9955526198316071
    Normalized spectral entropy of 2D data
    >>> np.random.seed(42)
    >>> x = np.random.normal(size=(4, 3000))
    >>> np.round(ent.spectral_entropy(x, sf=100, normalize=True), 4)
    array([0.9464, 0.9428, 0.9431, 0.9417])
    Fractional Gaussian noise with H = 0.5
    >>> import stochastic.processes.noise as sn
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.5, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.9505
    Fractional Gaussian noise with H = 0.9
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.9, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.8477
    Fractional Gaussian noise with H = 0.1
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.1, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.9248
    """
    x = np.asarray(x)
    #sf = np.array(sf, dtype=np.float64)
    # Compute and normalize power spectrum
    if method == 'fft':
        _, psd = periodogram(x, sf, axis=axis)
    elif method == 'welch':
        _, psd = welch(x, sf, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
    if normalize:
        se /= np.log2(psd_norm.shape[axis])
    return se

In [35]:
def statf(c):
    col = 'a_'+c
    mean = df[col].mean()
    var = df[col].var()
    std = df[col].std()
    mx = df[col].max()
    mn = df[col].min()
    rng = mx-mn
    #mode = df[col].mode()
    #n_z = ((df[col][:-1] * df[col][1:]) < 0).sum()
    
    f=abs(np.fft.fft(df[col]))
    num=df.shape[0]
    
    #fs=1./diff(t)
    
    freq = [i / num for i in list(range(num))]
    
    spectrum=f.real*f.real+f.imag*f.imag
    dc_comp = spectrum[0]
    spectrum = np.multiply(spectrum,spectrum)  # get the sectrum square
    spec_energy = spectrum.sum()/num
    
    # spectral entropy
    
    df['ts_shifted'] = df['ts'].shift(1)
    freq1=df['ts'] - df['ts_shifted']
    freq1 = freq1.dropna(axis = 0, how = 'all')
    mean_freq = np.mean(freq1)
    spec_entropy = spectral_entropy(df[col], mean_freq, method='fft', nperseg=None, normalize=False,axis=-1)
    
    # poer spectral density using LombScargle Periodogram
    
    frequency, power = LombScargle(df['ts'], df[col]).autopower()

    max_psd = np.max(power)
    min_psd = np.min(power)
    min_max_psd = min_psd/max_psd
    
    max_xas = np.sqrt(max_psd)
    min_xas = np.sqrt(min_psd)
    min_max_xas = min_xas/max_xas

    return np.array([mean,var,std,mx,mn,rng,dc_comp,spec_energy,spec_entropy, max_psd,min_psd,min_max_psd,max_xas,min_xas,min_max_xas])
    

In [47]:
data = np.array([])
sensor0 = np.array([])
sensor1 = np.array([])
sensor2 = np.array([])
sensor3 = np.array([])
sensor4 = np.array([])
sensor5 = np.array([])
sensor6 = np.array([])
sensor7 = np.array([])
sensor8 = np.array([])
sensor9 = np.array([])

k = 0

# for each sensor position
for i in range(10):
    
    sensor = np.array([])
    k = 0
    #for each activity
    for j in range(22):
        
        path = os.path.abspath(os.getcwd())+'\\sensor_csv'+'\\'+str(i)+'\\a'+str(j+1)
        files = os.listdir(path)
        
        # for each time seq
        for file in files:
            
            # namemaking
            #print(file)
            x = re.split("_|\.",file)
            name = x[0]
            time = x[1]
            mat_filename = name+"_a"+str(j+1)+"_"+time+".mat"
            filename = name+"_a"+str(j+1)+"_"+time
            
            df = pd.read_csv(path+'//'+file)
            x = statf('x')
            y = statf('y')
            z = statf('z')
            df['a_m'] =  np.sqrt(df['a_x']*df['a_x']+df['a_y']*df['a_y']+df['a_z']*df['a_z'])
            m = statf('m')
            act = j+1
            print(filename)
            
            l = [act,mat_filename,filename]
            sen = np.append(np.concatenate((x, y, z, m), axis = 0),l)                  
            # add all the features together to make a row
            
            #dat = np.append(np.concatenate((x, y, z, m), axis = 0),act)
            #dat = np.append(dat,i)
            
            # add all the new rows
            if(k==0):
                sensor = sen
            else: 
                sensor =  np.vstack([sensor,sen])
            k+=1
            #print(k)
    if(i==0):
        sensor0 = sensor
        print(len(sensor0))
    elif(i==1):
        sensor1 = sensor
    elif(i==2):
        sensor2 = sensor
    elif(i==3):
        sensor3 = sensor
    elif(i==4):
        sensor4 = sensor
    elif(i==5):
        sensor5 = sensor
    elif(i==6):
        sensor6 = sensor
    elif(i==7):
        sensor7 = sensor
    elif(i==8):
        sensor8 = sensor
    elif(i==9):
        sensor9 = sensor

cx_a1_t1
cx_a1_t10
cx_a1_t11
cx_a1_t12
cx_a1_t13
cx_a1_t2
cx_a1_t3
cx_a1_t4
cx_a1_t5
cx_a1_t6
cx_a1_t7
cx_a1_t8
cx_a1_t9
cyy_a1_t1
cyy_a1_t10
cyy_a1_t2
cyy_a1_t3
cyy_a1_t4
cyy_a1_t5
cyy_a1_t6
cyy_a1_t7
cyy_a1_t8
cyy_a1_t9
myj_a1_t1
myj_a1_t10
myj_a1_t2
myj_a1_t3
myj_a1_t4
myj_a1_t5
myj_a1_t6
myj_a1_t7
myj_a1_t8
myj_a1_t9
qyh_a1_t1
qyh_a1_t10
qyh_a1_t11
qyh_a1_t2
qyh_a1_t3
qyh_a1_t4
qyh_a1_t5
qyh_a1_t6
qyh_a1_t7
qyh_a1_t8
qyh_a1_t9
zyh_a1_t1
zyh_a1_t2
zyh_a1_t3
zyh_a1_t4
zyh_a1_t5
zyh_a1_t6
zyh_a1_t7
zyh_a1_t8
zyh_a1_t9
cx_a2_t1
cx_a2_t10
cx_a2_t11
cx_a2_t12
cx_a2_t13
cx_a2_t2
cx_a2_t3
cx_a2_t4
cx_a2_t5
cx_a2_t6
cx_a2_t7
cx_a2_t8
cx_a2_t9
cyy_a2_t1
cyy_a2_t10
cyy_a2_t2
cyy_a2_t3
cyy_a2_t4
cyy_a2_t5
cyy_a2_t6
cyy_a2_t7
cyy_a2_t8
cyy_a2_t9
myj_a2_t1
myj_a2_t10
myj_a2_t2
myj_a2_t3
myj_a2_t4
myj_a2_t5
myj_a2_t6
myj_a2_t7
myj_a2_t8
myj_a2_t9
qyh_a2_t1
qyh_a2_t10
qyh_a2_t11
qyh_a2_t2
qyh_a2_t3
qyh_a2_t4
qyh_a2_t5
qyh_a2_t6
qyh_a2_t7
qyh_a2_t8
qyh_a2_t9
zyh_a2_t1
zyh_a2_t2
zyh_a2_t3
zyh_a2_t4


cx_a16_t13
cx_a16_t2
cx_a16_t3
cx_a16_t4
cx_a16_t5
cx_a16_t6
cx_a16_t7
cx_a16_t8
cx_a16_t9
cyy_a16_t1
cyy_a16_t10
cyy_a16_t2
cyy_a16_t3
cyy_a16_t4
cyy_a16_t5
cyy_a16_t6
cyy_a16_t7
cyy_a16_t8
cyy_a16_t9
myj_a16_t1
myj_a16_t10
myj_a16_t2
myj_a16_t3
myj_a16_t4
myj_a16_t5
myj_a16_t6
myj_a16_t7
myj_a16_t8
myj_a16_t9
qyh_a16_t1
qyh_a16_t10
qyh_a16_t11
qyh_a16_t2
qyh_a16_t3
qyh_a16_t4
qyh_a16_t5
qyh_a16_t6
qyh_a16_t7
qyh_a16_t8
qyh_a16_t9
zyh_a16_t1
zyh_a16_t2
zyh_a16_t3
zyh_a16_t4
zyh_a16_t5
zyh_a16_t6
zyh_a16_t7
zyh_a16_t8
zyh_a16_t9
cx_a17_t1
cx_a17_t10
cx_a17_t11
cx_a17_t12
cx_a17_t13
cx_a17_t2
cx_a17_t3
cx_a17_t4
cx_a17_t5
cx_a17_t6
cx_a17_t7
cx_a17_t8
cx_a17_t9
cyy_a17_t1
cyy_a17_t10
cyy_a17_t2
cyy_a17_t3
cyy_a17_t4
cyy_a17_t5
cyy_a17_t6
cyy_a17_t7
cyy_a17_t8
cyy_a17_t9
myj_a17_t1
myj_a17_t10
myj_a17_t2
myj_a17_t3
myj_a17_t4
myj_a17_t5
myj_a17_t6
myj_a17_t7
myj_a17_t8
myj_a17_t9
qyh_a17_t1
qyh_a17_t10
qyh_a17_t11
qyh_a17_t2
qyh_a17_t3
qyh_a17_t4
qyh_a17_t5
qyh_a17_t6
qyh_a17_t7
qyh_a17_

<ipython-input-34-b64774160073>:88: RuntimeWarning: divide by zero encountered in log2
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
<ipython-input-34-b64774160073>:88: RuntimeWarning: invalid value encountered in multiply
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)


cx_a5_t12
cx_a5_t13
cx_a5_t2
cx_a5_t3
cx_a5_t4
cx_a5_t5
cx_a5_t6
cx_a5_t7
cx_a5_t8
cx_a5_t9
cyy_a5_t1
cyy_a5_t10
cyy_a5_t2
cyy_a5_t3
cyy_a5_t4
cyy_a5_t5
cyy_a5_t6
cyy_a5_t7
cyy_a5_t8
cyy_a5_t9
myj_a5_t1
myj_a5_t10
myj_a5_t2
myj_a5_t3
myj_a5_t4
myj_a5_t5
myj_a5_t6
myj_a5_t7
myj_a5_t8
myj_a5_t9
qyh_a5_t1
qyh_a5_t10
qyh_a5_t11
qyh_a5_t2
qyh_a5_t3
qyh_a5_t4
qyh_a5_t5
qyh_a5_t6
qyh_a5_t7
qyh_a5_t8
qyh_a5_t9
zyh_a5_t1
zyh_a5_t2
zyh_a5_t3
zyh_a5_t4
zyh_a5_t5
zyh_a5_t6
zyh_a5_t7
zyh_a5_t8
zyh_a5_t9
cx_a6_t1
cx_a6_t10
cx_a6_t11
cx_a6_t12
cx_a6_t13
cx_a6_t2
cx_a6_t3
cx_a6_t4
cx_a6_t5
cx_a6_t6
cx_a6_t7
cx_a6_t8
cx_a6_t9
cyy_a6_t1
cyy_a6_t10
cyy_a6_t2
cyy_a6_t3
cyy_a6_t4
cyy_a6_t5
cyy_a6_t6
cyy_a6_t7
cyy_a6_t8
cyy_a6_t9
myj_a6_t1
myj_a6_t10
myj_a6_t2
myj_a6_t3
myj_a6_t4
myj_a6_t5
myj_a6_t6
myj_a6_t7
myj_a6_t8
myj_a6_t9
qyh_a6_t1
qyh_a6_t10
qyh_a6_t11
qyh_a6_t2
qyh_a6_t3
qyh_a6_t4
qyh_a6_t5
qyh_a6_t6
qyh_a6_t7
qyh_a6_t8
qyh_a6_t9
zyh_a6_t1
zyh_a6_t2
zyh_a6_t3
zyh_a6_t4
zyh_a6_t5
zyh_a6_t6
zyh_a6_t7

qyh_a19_t4
qyh_a19_t5
qyh_a19_t6
qyh_a19_t7
qyh_a19_t8
qyh_a19_t9
zyh_a19_t1
zyh_a19_t2
zyh_a19_t3
zyh_a19_t4
zyh_a19_t5
zyh_a19_t6
zyh_a19_t7
zyh_a19_t8
zyh_a19_t9
cx_a20_t1
cx_a20_t10
cx_a20_t11
cx_a20_t12
cx_a20_t13
cx_a20_t2
cx_a20_t3
cx_a20_t4
cx_a20_t5
cx_a20_t6
cx_a20_t7
cx_a20_t8
cx_a20_t9
cyy_a20_t1
cyy_a20_t10
cyy_a20_t2
cyy_a20_t3
cyy_a20_t4
cyy_a20_t5
cyy_a20_t6
cyy_a20_t7
cyy_a20_t8
cyy_a20_t9
myj_a20_t1
myj_a20_t10
myj_a20_t2
myj_a20_t3
myj_a20_t4
myj_a20_t5
myj_a20_t6
myj_a20_t7
myj_a20_t8
myj_a20_t9
qyh_a20_t1
qyh_a20_t10
qyh_a20_t11
qyh_a20_t2
qyh_a20_t3
qyh_a20_t4
qyh_a20_t5
qyh_a20_t6
qyh_a20_t7
qyh_a20_t8
qyh_a20_t9
zyh_a20_t1
zyh_a20_t2
zyh_a20_t3
zyh_a20_t4
zyh_a20_t5
zyh_a20_t6
zyh_a20_t7
zyh_a20_t8
zyh_a20_t9
cx_a21_t1
cx_a21_t10
cx_a21_t11
cx_a21_t12
cx_a21_t13
cx_a21_t2
cx_a21_t3
cx_a21_t4
cx_a21_t5
cx_a21_t6
cx_a21_t7
cx_a21_t8
cx_a21_t9
cyy_a21_t1
cyy_a21_t10
cyy_a21_t2
cyy_a21_t3
cyy_a21_t4
cyy_a21_t5
cyy_a21_t6
cyy_a21_t7
cyy_a21_t8
cyy_a21_t9
myj_a21_t1
m

...

cx_a6_t10
cx_a6_t11
cx_a6_t12
cx_a6_t13
cx_a6_t2
cx_a6_t3
cx_a6_t4
cx_a6_t5
cx_a6_t6
cx_a6_t7
cx_a6_t8
cx_a6_t9
cyy_a6_t1
cyy_a6_t10
cyy_a6_t2
cyy_a6_t3
cyy_a6_t4
cyy_a6_t5
cyy_a6_t6
cyy_a6_t7
cyy_a6_t8
cyy_a6_t9
myj_a6_t1
myj_a6_t10
myj_a6_t2
myj_a6_t3
myj_a6_t4
myj_a6_t5
myj_a6_t6
myj_a6_t7
myj_a6_t8
myj_a6_t9
qyh_a6_t1
qyh_a6_t10
qyh_a6_t11
qyh_a6_t2
qyh_a6_t3
qyh_a6_t4
qyh_a6_t5
qyh_a6_t6
qyh_a6_t7
qyh_a6_t8
qyh_a6_t9
zyh_a6_t1
zyh_a6_t2
zyh_a6_t3
zyh_a6_t4
zyh_a6_t5
zyh_a6_t6
zyh_a6_t7
zyh_a6_t8
zyh_a6_t9
cx_a7_t1
cx_a7_t10
cx_a7_t11
cx_a7_t12
cx_a7_t13
cx_a7_t2
cx_a7_t3
cx_a7_t4
cx_a7_t5
cx_a7_t6
cx_a7_t7
cx_a7_t8
cx_a7_t9
cyy_a7_t1
cyy_a7_t10
cyy_a7_t2
cyy_a7_t3
cyy_a7_t4
cyy_a7_t5
cyy_a7_t6
cyy_a7_t7
cyy_a7_t8
cyy_a7_t9
myj_a7_t1
myj_a7_t10
myj_a7_t2
myj_a7_t3
myj_a7_t4
myj_a7_t5
myj_a7_t6
myj_a7_t7
myj_a7_t8
myj_a7_t9
qyh_a7_t1
qyh_a7_t10
qyh_a7_t11
qyh_a7_t2
qyh_a7_t3
qyh_a7_t4
qyh_a7_t5
qyh_a7_t6
qyh_a7_t7
qyh_a7_t8
qyh_a7_t9
zyh_a7_t1
zyh_a7_t2
zyh_a7_t3
zyh_a7_t4
zyh_a7_t5

myj_a20_t8
myj_a20_t9
qyh_a20_t1
qyh_a20_t10
qyh_a20_t11
qyh_a20_t2
qyh_a20_t3
qyh_a20_t4
qyh_a20_t5
qyh_a20_t6
qyh_a20_t7
qyh_a20_t8
qyh_a20_t9
zyh_a20_t1
zyh_a20_t2
zyh_a20_t3
zyh_a20_t4
zyh_a20_t5
zyh_a20_t6
zyh_a20_t7
zyh_a20_t8
zyh_a20_t9
cx_a21_t1
cx_a21_t10
cx_a21_t11
cx_a21_t12
cx_a21_t13
cx_a21_t2
cx_a21_t3
cx_a21_t4
cx_a21_t5
cx_a21_t6
cx_a21_t7
cx_a21_t8
cx_a21_t9
cyy_a21_t1
cyy_a21_t10
cyy_a21_t2
cyy_a21_t3
cyy_a21_t4
cyy_a21_t5
cyy_a21_t6
cyy_a21_t7
cyy_a21_t8
cyy_a21_t9
myj_a21_t1
myj_a21_t10
myj_a21_t2
myj_a21_t3
myj_a21_t4
myj_a21_t5
myj_a21_t6
myj_a21_t7
myj_a21_t8
myj_a21_t9
qyh_a21_t1
qyh_a21_t10
qyh_a21_t11
qyh_a21_t2
qyh_a21_t3
qyh_a21_t4
qyh_a21_t5
qyh_a21_t6
qyh_a21_t7
qyh_a21_t8
qyh_a21_t9
zyh_a21_t1
zyh_a21_t2
zyh_a21_t3
zyh_a21_t4
zyh_a21_t5
zyh_a21_t6
zyh_a21_t7
zyh_a21_t8
zyh_a21_t9
cx_a22_t1
cx_a22_t10
cx_a22_t11
cx_a22_t12
cx_a22_t13
cx_a22_t2
cx_a22_t3
cx_a22_t4
cx_a22_t5
cx_a22_t6
cx_a22_t7
cx_a22_t8
cx_a22_t9
cyy_a22_t1
cyy_a22_t10
cyy_a22_t2
cyy_a22_t3

qyh_a13_t1
qyh_a13_t10
qyh_a13_t11
qyh_a13_t2
qyh_a13_t3
qyh_a13_t4
qyh_a13_t5
qyh_a13_t6
qyh_a13_t7
qyh_a13_t8
qyh_a13_t9
zyh_a13_t1
zyh_a13_t2
zyh_a13_t3
zyh_a13_t4
zyh_a13_t5
zyh_a13_t6
zyh_a13_t7
zyh_a13_t8
zyh_a13_t9
cx_a14_t1
cx_a14_t10
cx_a14_t11
cx_a14_t12
cx_a14_t13
cx_a14_t2
cx_a14_t3
cx_a14_t4
cx_a14_t5
cx_a14_t6
cx_a14_t7
cx_a14_t8
cx_a14_t9
cyy_a14_t1
cyy_a14_t10
cyy_a14_t2
cyy_a14_t3
cyy_a14_t4
cyy_a14_t5
cyy_a14_t6
cyy_a14_t7
cyy_a14_t8
cyy_a14_t9
myj_a14_t1
myj_a14_t10
myj_a14_t2
myj_a14_t3
myj_a14_t4
myj_a14_t5
myj_a14_t6
myj_a14_t7
myj_a14_t8
myj_a14_t9
qyh_a14_t1
qyh_a14_t10
qyh_a14_t11
qyh_a14_t2
qyh_a14_t3
qyh_a14_t4
qyh_a14_t5
qyh_a14_t6
qyh_a14_t7
qyh_a14_t8
qyh_a14_t9
zyh_a14_t1
zyh_a14_t2
zyh_a14_t3
zyh_a14_t4
zyh_a14_t5
zyh_a14_t6
zyh_a14_t7
zyh_a14_t8
zyh_a14_t9
cx_a15_t1
cx_a15_t10
cx_a15_t11
cx_a15_t12
cx_a15_t13
cx_a15_t2
cx_a15_t3
cx_a15_t4
cx_a15_t5
cx_a15_t6
cx_a15_t7
cx_a15_t8
cx_a15_t9
cyy_a15_t1
cyy_a15_t10
cyy_a15_t2
cyy_a15_t3
cyy_a15_t4
cyy_a15_t5

cyy_a6_t3
cyy_a6_t4
cyy_a6_t5
cyy_a6_t6
cyy_a6_t7
cyy_a6_t8
cyy_a6_t9
myj_a6_t1
myj_a6_t10
myj_a6_t2
myj_a6_t3
myj_a6_t4
myj_a6_t5
myj_a6_t6
myj_a6_t7
myj_a6_t8
myj_a6_t9
qyh_a6_t1
qyh_a6_t10
qyh_a6_t11
qyh_a6_t2
qyh_a6_t3
qyh_a6_t4
qyh_a6_t5
qyh_a6_t6
qyh_a6_t7
qyh_a6_t8
qyh_a6_t9
zyh_a6_t1
zyh_a6_t2
zyh_a6_t3
zyh_a6_t4
zyh_a6_t5
zyh_a6_t6
zyh_a6_t7
zyh_a6_t8
zyh_a6_t9
cx_a7_t1
cx_a7_t10
cx_a7_t11
cx_a7_t12
cx_a7_t13
cx_a7_t2
cx_a7_t3
cx_a7_t4
cx_a7_t5
cx_a7_t6
cx_a7_t7
cx_a7_t8
cx_a7_t9
cyy_a7_t1
cyy_a7_t10
cyy_a7_t2
cyy_a7_t3
cyy_a7_t4
cyy_a7_t5
cyy_a7_t6
cyy_a7_t7
cyy_a7_t8
cyy_a7_t9
myj_a7_t1
myj_a7_t10
myj_a7_t2
myj_a7_t3
myj_a7_t4
myj_a7_t5
myj_a7_t6
myj_a7_t7
myj_a7_t8
myj_a7_t9
qyh_a7_t1
qyh_a7_t10
qyh_a7_t11
qyh_a7_t2
qyh_a7_t3
qyh_a7_t4
qyh_a7_t5
qyh_a7_t6
qyh_a7_t7
qyh_a7_t8
qyh_a7_t9
zyh_a7_t1
zyh_a7_t2
zyh_a7_t3
zyh_a7_t4
zyh_a7_t5
zyh_a7_t6
zyh_a7_t7
zyh_a7_t8
zyh_a7_t9
cx_a8_t1
cx_a8_t10
cx_a8_t11
cx_a8_t12
cx_a8_t13
cx_a8_t2
cx_a8_t3
cx_a8_t4
cx_a8_t5
cx_a8_t6
cx_a8_t7

zyh_a20_t1
zyh_a20_t2
zyh_a20_t3
zyh_a20_t4
zyh_a20_t5
zyh_a20_t6
zyh_a20_t7
zyh_a20_t8
zyh_a20_t9
cx_a21_t1
cx_a21_t10
cx_a21_t11
cx_a21_t12
cx_a21_t13
cx_a21_t2
cx_a21_t3
cx_a21_t4
cx_a21_t5
cx_a21_t6
cx_a21_t7
cx_a21_t8
cx_a21_t9
cyy_a21_t1
cyy_a21_t10
cyy_a21_t2
cyy_a21_t3
cyy_a21_t4
cyy_a21_t5
cyy_a21_t6
cyy_a21_t7
cyy_a21_t8
cyy_a21_t9
myj_a21_t1
myj_a21_t10
myj_a21_t2
myj_a21_t3
myj_a21_t4
myj_a21_t5
myj_a21_t6
myj_a21_t7
myj_a21_t8
myj_a21_t9
qyh_a21_t1
qyh_a21_t10
qyh_a21_t11
qyh_a21_t2
qyh_a21_t3
qyh_a21_t4
qyh_a21_t5
qyh_a21_t6
qyh_a21_t7
qyh_a21_t8
qyh_a21_t9
zyh_a21_t1
zyh_a21_t2
zyh_a21_t3
zyh_a21_t4
zyh_a21_t5
zyh_a21_t6
zyh_a21_t7
zyh_a21_t8
zyh_a21_t9
cx_a22_t1
cx_a22_t10
cx_a22_t11
cx_a22_t12
cx_a22_t13
cx_a22_t2
cx_a22_t3
cx_a22_t4
cx_a22_t5
cx_a22_t6
cx_a22_t7
cx_a22_t8
cx_a22_t9
cyy_a22_t1
cyy_a22_t10
cyy_a22_t2
cyy_a22_t3
cyy_a22_t4
cyy_a22_t5
cyy_a22_t6
cyy_a22_t7
cyy_a22_t8
cyy_a22_t9
myj_a22_t1
myj_a22_t10
myj_a22_t2
myj_a22_t3
myj_a22_t4
myj_a22_t5
myj_a22_t6


In [48]:
print(len(sensor0))
print(len(sensor1))
print(len(sensor2))
print(len(sensor3))
print(len(sensor4))
print(len(sensor5))
print(len(sensor6))
print(len(sensor7))
print(len(sensor7))
print(len(sensor8))

1165
1165
1165
1165
1165
1165
1165
1165
1165
1165


In [49]:
sensor = sensor0

sensor = np.concatenate([sensor,sensor1],axis=1)
sensor = np.concatenate([sensor,sensor2],axis=1)
sensor = np.concatenate([sensor,sensor3],axis=1)
sensor = np.concatenate([sensor,sensor4],axis=1)
sensor = np.concatenate([sensor,sensor5],axis=1)
sensor = np.concatenate([sensor,sensor6],axis=1)
sensor = np.concatenate([sensor,sensor7],axis=1)
sensor = np.concatenate([sensor,sensor8],axis=1)
sensor = np.concatenate([sensor,sensor9],axis=1)

In [50]:
sensor.shape

(1165, 630)

In [67]:

# Python program to iterate
# over 3 lists using zip function
 
import itertools

In [73]:
sen  = ['0_','1_','2_','3_','4_','5_','6_','7_','8_','9_']

# mean,var,std,mx,mn,rng,dc_comp,spec_energy,spec_entropy, max_psd,min_psd,min_max_psd,max_xas,min_xas,min_max_xas

col = ['mean','var','std','max','min','range','dc_comp','spec_energy','spec_entropy','max_psd','min_psd','min_max_psd','max_xas','min_xas','min_max_xas']
axis = ['_x','_y','_z','_m']
col1 = ['activity','mat_filename','filename']

col_name = []
for s in sen:
    for (c) in (col):
        for x in axis:
            col_name.append(s+c+x)
    for c1 in col1:
        col_name.append(s+c1)
    
    
# col_name.append('activity')
# col_name.append('position')
print(col_name)

['0_mean_x', '0_mean_y', '0_mean_z', '0_mean_m', '0_var_x', '0_var_y', '0_var_z', '0_var_m', '0_std_x', '0_std_y', '0_std_z', '0_std_m', '0_max_x', '0_max_y', '0_max_z', '0_max_m', '0_min_x', '0_min_y', '0_min_z', '0_min_m', '0_range_x', '0_range_y', '0_range_z', '0_range_m', '0_dc_comp_x', '0_dc_comp_y', '0_dc_comp_z', '0_dc_comp_m', '0_spec_energy_x', '0_spec_energy_y', '0_spec_energy_z', '0_spec_energy_m', '0_spec_entropy_x', '0_spec_entropy_y', '0_spec_entropy_z', '0_spec_entropy_m', '0_max_psd_x', '0_max_psd_y', '0_max_psd_z', '0_max_psd_m', '0_min_psd_x', '0_min_psd_y', '0_min_psd_z', '0_min_psd_m', '0_min_max_psd_x', '0_min_max_psd_y', '0_min_max_psd_z', '0_min_max_psd_m', '0_max_xas_x', '0_max_xas_y', '0_max_xas_z', '0_max_xas_m', '0_min_xas_x', '0_min_xas_y', '0_min_xas_z', '0_min_xas_m', '0_min_max_xas_x', '0_min_max_xas_y', '0_min_max_xas_z', '0_min_max_xas_m', '0_activity', '0_mat_filename', '0_filename', '1_mean_x', '1_mean_y', '1_mean_z', '1_mean_m', '1_var_x', '1_var_y',

In [74]:
len(col_name)

630

In [75]:
xD = pd.DataFrame(sensor)
xD.columns = col_name

In [76]:
act = xD['8_activity']
act2 = xD['0_activity']

In [87]:
# print(act)
# print(act2)
# print(act==act2)

In [79]:
xD.to_csv('sensor_data_preprocessed.csv', index=False)

In [83]:
mat_fn = xD['9_mat_filename']
fn = xD['9_filename']

In [91]:
drop_list = []
for s in sen:
    for c1 in col1:
        drop_list.append(s+c1)
print(drop_list)
xD = xD.drop(drop_list, axis=1)

['0_activity', '0_mat_filename', '0_filename', '1_activity', '1_mat_filename', '1_filename', '2_activity', '2_mat_filename', '2_filename', '3_activity', '3_mat_filename', '3_filename', '4_activity', '4_mat_filename', '4_filename', '5_activity', '5_mat_filename', '5_filename', '6_activity', '6_mat_filename', '6_filename', '7_activity', '7_mat_filename', '7_filename', '8_activity', '8_mat_filename', '8_filename', '9_activity', '9_mat_filename', '9_filename']


In [92]:
xD['mat_filename'] = mat_fn
xD['filename'] = fn
xD['activity'] = act

In [93]:
xD.head()

,0_mean_x,0_mean_y,0_mean_z,0_mean_m,0_var_x,0_var_y,0_var_z,0_var_m,0_std_x,0_std_y,...,9_min_xas_y,9_min_xas_z,9_min_xas_m,9_min_max_xas_x,9_min_max_xas_y,9_min_max_xas_z,9_min_max_xas_m,mat_filename,filename,activity
0,0.08440677527151225,0.005099253183426285,0.07140905533212356,0.24560500000000002,-0.108154,0.35375900000000005,40832.2121548324,738671.4967301019,3.2067189675424275,0.4728212690903644,...,7.534361137953364,0.12216971247063237,4.31148659960132e-09,3.529096133902854e-08,0.34952784219663013,6.566191133070465e-05,0.0001878588867715034,cx_a1_t1.mat,cx_a1_t1,1
1,0.05868655153061222,0.00557284670697336,0.07465150170608331,0.238037,-0.14868199999999998,0.386719,13230.898087460882,132037.6017507014,2.8378307843951722,0.6569401400238601,...,7.667917190660498,0.03191200126246252,1.1211442148814118e-09,3.513236934470144e-08,0.17863930492045282,3.348349167696541e-05,0.0001874363074345561,cx_a1_t10.mat,cx_a1_t10,1
2,0.09736474987133306,0.005283285950662429,0.07268621568538583,0.280518,-0.11377000000000001,0.39428799999999997,35788.962297324695,701134.9325655709,2.7171058541770092,0.6296151794730955,...,8.028464777990097,0.023385442451643996,1.8079883052355656e-09,7.73125549783414e-08,0.15292299516960814,4.252044573185429e-05,0.0002780513531316498,cx_a1_t11.mat,cx_a1_t11,1
3,0.06759654350927238,0.0037317322946421347,0.06108790628792359,0.17871099999999998,-0.13623,0.31494099999999997,20208.19499389196,211783.71469614343,3.357616831031328,0.5779111521277593,...,7.83468015334643,0.07270179303544692,1.740906185286625e-09,2.394584937455144e-08,0.2696327002339422,4.172416788009828e-05,0.00015474446476223774,cx_a1_t12.mat,cx_a1_t12,1
4,0.07944031100000015,0.005409067411984266,0.07354636233005862,0.222412,-0.09887699999999999,0.321289,25243.05204710689,374472.95690838876,2.214226934810065,0.7474342064044721,...,7.879593052127465,0.0652934821293403,7.301426620893046e-09,1.1182473935805109e-07,0.2555258932659082,8.544838571262213e-05,0.0003344020624309173,cx_a1_t13.mat,cx_a1_t13,1


In [94]:
xD.to_csv('sensor_data_preprocessed_new.csv', index=False)